# ratios in numerical columns

there is an interesting outcome when table cells have access to their `--ratio` in the distribution available in css. these values make it possible to use visual forms to identify colors. it is potentially an axis of design to experiment to [design for low numeracy](https://accessiblenumbers.com/).

In [1]:
%%
{{table._repr_html_()}}

    gists = (await (
        "https://api.github.com/users/tonyfast/gists?page=" + pandas.RangeIndex(1, 3).astype(str)
    ).http.get()).explode().series().set_index("id")
    gists.index.name = "gist_id"
we're going to quickly make a table of my gist from github.

    files = gists.join(gists.files.apply(dict.values).apply(list).explode().series())
unravel all the gists in each gist payload.

    time = files.columns[files.columns.str.endswith("_at")].tolist()
    files[time] = files[time].apply(pandas.to_datetime, axis=1)
    files = files.assign(timespan=files.updated_at - files.created_at)

convert times to their proper types and substract them so out dataframe 
contains integers, datetimes, and timedeltas, types with numerical representations.
    
    files[cat] = files[cat := "language type".split()].astype("category")
    df = files[time + "timespan filename description language type size".split()]
    
now are going to add a few lines of `css` that uses the `--ratio` variables in each numerical cell
to produce visual representation of the cell value in the larger distribution.
it is really interesting how we start to create the cell border form when we do this.
at a glance we can visually observe relative positions of values in a column.
further, scrolling creates really interesting movement that provides user controlled animation.
    
    display\
```css
td {
    --marker-width: .2ch;
    background: linear-gradient(90deg, 
                                rgba(255, 0, 0, 0) 0% calc(var(--ratio) * 100% - var(--marker-width)),
                                rgba(255, 0, 0, 1) calc(var(--ratio) * 100% - var(--marker-width)) calc(var(--ratio) * 100% + var(--marker-width)),
                                rgba(255, 0, 0, 0) calc(var(--ratio) * 100% + var(--marker-width)) 100%
                               ),
                linear-gradient(0deg, 
                                rgba(255, 0, 0, 0) 0% calc(var(--ratio) * 100% - var(--marker-width)),
                                rgba(255, 0, 0, 1) calc(var(--ratio) * 100% - var(--marker-width)) calc(var(--ratio) * 100% + var(--marker-width)),
                                rgba(255, 0, 0, 0) calc(var(--ratio) * 100% + var(--marker-width)) 100%
                               ); 
}
```

    notebooks = (
        await files[files.language.eq("Jupyter Notebook")].raw_url.http.get()
    ).apply(json.loads).series()
read the `notebooks` and separate out the cells, outputs, and display objects.

    cells = notebooks.cells.enumerate("cell").series()
    outputs = cells.outputs[cells.outputs.fillna("").astype(bool)].enumerate("display").series()
    display_data = outputs['data'].dropna().series()

    df = (df.join(cells.groupby("gist_id").id.count().rename("cells"))
          .join(outputs.groupby("gist_id")["data"].count().rename("outputs")))
    import nbconvert_a11y.table
    table = df.sort_values("created_at", ascending=False).table(id="gist")

'td {\n    --marker-width: .2ch;\n    background: linear-gradient(90deg, \n                                rgba(255, 0, 0, 0) 0% calc(var(--ratio) * 100% - var(--marker-width)),\n                                rgba(255, 0, 0, 1) calc(var(--ratio) * 100% - var(--marker-width)) calc(var(--ratio) * 100% + var(--marker-width)),\n                                rgba(255, 0, 0, 0) calc(var(--ratio) * 100% + var(--marker-width)) 100%\n                               ),\n                linear-gradient(0deg, \n                                rgba(255, 0, 0, 0) 0% calc(var(--ratio) * 100% - var(--marker-width)),\n                                rgba(255, 0, 0, 1) calc(var(--ratio) * 100% - var(--marker-width)) calc(var(--ratio) * 100% + var(--marker-width)),\n                                rgba(255, 0, 0, 0) calc(var(--ratio) * 100% + var(--marker-width)) 100%\n                               ); \n}\n'

gist_id,created_at,updated_at,timespan,filename,description,language,type,size,cells,outputs
90c41d4994f75c594db804aeba56fc26,2024-01-10T21:04:06+00:00,2024-01-10T21:04:39+00:00,33.0 s,first_and_second_laws_of_thermodynamics.ipynb,a notebook with revised literate pidgy cells for the first part of thermodyamics course,Jupyter Notebook,text/plain,20469,16.0,7.0
aa3b16c5a284150e3d727a843b6cefec,2023-12-03T21:31:06+00:00,2023-12-03T21:31:06+00:00,0.0 s,axe_types.py,,Python,application/x-python,8442,nan,nan
713ae6c57602c0f85d011421b20d5ea0,2023-11-02T20:59:23+00:00,2023-11-02T21:05:32+00:00,369.0 s,BinaryExamples.ipynb,a revision of a pycalphad documentation page as literate program.,Jupyter Notebook,text/plain,352154,0.0,6.0
c004044b4fe641735031ecf2069cf595,2023-06-06T18:09:18+00:00,2023-06-06T18:09:19+00:00,1.0 s,aom.json,an exported accessibility tree from https://iota-school.github.io/notebooks-for-all/branch/extent/exports/html/lorenz-executed-smol.html#14,JSON,application/json,52330,nan,nan
e17946facd998a931527467d646cc822,2023-02-21T22:32:31+00:00,2023-02-21T22:35:44+00:00,193.0 s,README.md,extensible notebook schemas,Markdown,text/markdown,3044,nan,nan
99303420edc6021bbeeaa96c522469a1,2022-12-23T06:16:30+00:00,2022-12-23T06:16:30+00:00,0.0 s,ummm.py,,Python,application/x-python,266,nan,nan
c8e19ffb2b85f5bf6f5a4b6e76c327cd,2022-12-20T03:08:25+00:00,2022-12-20T03:08:25+00:00,0.0 s,abc.py,abc of python modules,Python,application/x-python,469,nan,nan
4e33098a9f3a50805ad99d08e96825a6,2022-12-18T21:07:29+00:00,2022-12-18T21:07:29+00:00,0.0 s,graph shit.py,,Python,application/x-python,4213,nan,nan
243d7758d659c406f6e12cbd7b045384,2022-12-17T23:08:41+00:00,2022-12-17T23:08:41+00:00,0.0 s,hide_empty_outputs.html,a selection to hide empty jupyterlab output cells,HTML,text/html,90,nan,nan
42612252e0ed0df02f811932becd8241,2022-12-02T18:45:11+00:00,2022-12-02T18:45:11+00:00,0.0 s,2022-12-01-1.ipynb,,Jupyter Notebook,text/plain,4956,8.0,0.0


* the type row needs to get added
* experiment with different cells markers and css gradients

## marginalia


i added `aria-sort` annotations to the table and thought some css would give visual representation of the sortedness.
add much context and information will facilitate an accessible experience dyscalculia.

```css
th[aria-sort]::after {
    content: attr(aria-sort);
    font-weight: 100;
}

```